##prec/Recall

In [40]:
import itertools

class Classification:
    TruePos, FalsePos, TrueNeg, FalseNeg = range(4)

def classType(true1, true2, pred1, pred2):
    if true1 == true2:
        if pred1 == pred2:
            return Classification.TruePos
        else: # truely the same, predicted different
            return Classification.FalseNeg
    else: # truly different
        if pred1 == pred2: #predicted same
            return Classification.FalsePos
        else:
            return Classification.TrueNeg
        
    
def accuracyExpressed(groundTruth_clust, tr_clust):
    #count true postive for each pair of transcripts O(N^2)
    tp, fp, tn, fn = 0, 0, 0, 0
    for tr_1, tr_2 in itertools.combinations(tr_clust.keys(), 2):
        if tr_1 not in groundTruth_clust or tr_2 not in groundTruth_clust:
            continue
        ct = classType(groundTruth_clust[tr_1], groundTruth_clust[tr_2], tr_clust[tr_1], tr_clust[tr_2]) 
        if ct == Classification.TruePos:
            tp += 1
        elif ct == Classification.TrueNeg:
            tn += 1
        elif ct == Classification.FalsePos:
            fp += 1
        elif ct == Classification.FalseNeg:
            fn += 1
    return tp, fp, tn, fn


def accuracyExpressedFast(groundTruth_clust, groundTruth_clust_inv,
                          tr_clust, tr_clust_inv):
    num = len(set(tr_clust.keys()) & set(groundTruth_clust.keys()))
    tp, fp, tn, fn = 0, 0, 0, 0
    for clustName, clustMems in tr_clust_inv.iteritems():
        for tr_1, tr_2 in itertools.combinations(clustMems,2):
            if tr_1 not in groundTruth_clust or tr_2 not in groundTruth_clust:
                continue
            if groundTruth_clust[tr_1] == groundTruth_clust[tr_2]:
                tp += 1
            else:
                fp += 1
    for clustName, clustMems in groundTruth_clust_inv.iteritems():
        for tr_1, tr_2 in itertools.combinations(clustMems,2):
            if tr_1 not in tr_clust or tr_2 not in tr_clust:
                continue
            if tr_clust[tr_1] != tr_clust[tr_2]:
                fn += 1
    nc2 = (num * (num-1)) / 2
    tn = nc2 - (fp + tp + fn)
    return tp, fp, tn, fn

def readCDHitClust(fn, filtDict=None):
    tr_clust = {}
    tr_clust_inv = {}
    fp = open(fn)
    cnum = None
    for l in fp:
        if l.startswith('>Cluster'):
            cnum = int(l.rstrip().split()[-1])
        else:
            e = l.split()[2].lstrip('>').rstrip('.')
            if not filtDict or e in filtDict:
                tr_clust[e] = cnum
    for k,v in tr_clust.iteritems():
        if v in tr_clust_inv:
            tr_clust_inv[v].append(k)
        else:
            tr_clust_inv[v] = [k]
    return tr_clust, tr_clust_inv

def readCorset(fn):
    ft = open(fn)
    clust_dict = {}
    clust_dict_inv = {}
    for line in ft:
        toks = line.rstrip().split()
        clust_dict[toks[0]] = toks[1]
    for k,v in clust_dict.iteritems():
        if v in clust_dict_inv:
            clust_dict_inv[v].append(k)
        else:
            clust_dict_inv[v] = [k]
    return clust_dict, clust_dict_inv

##CD-HIT-0.95

In [46]:
ft = "/mnt/scratch3/avi/clustering/human/cd_hit/0.95.clstr"
tr_clust, tr_clust_inv = readCDHitClust(ft)
ft = "/mnt/scratch3/avi/clustering/human/truth/contigs2genes.disambiguous.txt"
groundTruth_clust, ground_truth_clust_inv = readTrueLabels(ft)
tp, fp, tn, fn = accuracyExpressedFast(groundTruth_clust, ground_truth_clust_inv, tr_clust, tr_clust_inv)
print("tp : {}, fp : {}, tn : {}, fn : {}".format(tp, fp, tn, fn))
print("prec: {}, recall: {}".format(tp / float(tp + fp), tp / float(tp + fn)))

tp : 21499, fp : 405, tn : 958295803, fn : 135946
prec: 0.981510226443, recall: 0.136549271174


##CD-HIT-0.8

In [47]:
ft = "/mnt/scratch3/avi/clustering/human/cd_hit/0.8.clstr"
tr_clust, tr_clust_inv = readCDHitClust(ft)
ft = "/mnt/scratch3/avi/clustering/human/truth/contigs2genes.disambiguous.txt"
groundTruth_clust, ground_truth_clust_inv = readTrueLabels(ft)
tp, fp, tn, fn = accuracyExpressedFast(groundTruth_clust, ground_truth_clust_inv, tr_clust, tr_clust_inv)
print("tp : {}, fp : {}, tn : {}, fn : {}".format(tp, fp, tn, fn))
print("prec: {}, recall: {}".format(tp / float(tp + fp), tp / float(tp + fn)))

tp : 58979, fp : 2607, tn : 958293601, fn : 98466
prec: 0.957668950736, recall: 0.374600654197


##corset

In [49]:
ft = "/mnt/scratch3/avi/clustering/corsetData/Human-Trinity/corset-clusters.txt"
tr_clust, tr_clust_inv = readCorset(ft)
ft = "/mnt/scratch3/avi/clustering/human/truth/contigs2genes.disambiguous.txt"
groundTruth_clust, ground_truth_clust_inv = readTrueLabels(ft)
tp, fp, tn, fn = accuracyExpressedFast(groundTruth_clust, ground_truth_clust_inv, tr_clust, tr_clust_inv)
print("tp : {}, fp : {}, tn : {}, fn : {}".format(tp, fp, tn, fn))
print("prec: {}, recall: {}".format(tp / float(tp + fp), tp / float(tp + fn)))

tp : 88003, fp : 3921, tn : 958292287, fn : 69442
prec: 0.957345198207, recall: 0.55894439328


##qsf

In [50]:
ft = "/home/hirak/rapmapClustering/sailfish/build/time_results/quant_human.clust"
tr_clust, tr_clust_inv = readMCLClust(ft)
ft = "/mnt/scratch3/avi/clustering/human/truth/contigs2genes.disambiguous.txt"
groundTruth_clust, ground_truth_clust_inv = readTrueLabels(ft)
tp, fp, tn, fn = accuracyExpressedFast(groundTruth_clust, ground_truth_clust_inv, tr_clust, tr_clust_inv)
print("tp : {}, fp : {}, tn : {}, fn : {}".format(tp, fp, tn, fn))
print("prec: {}, recall: {}".format(tp / float(tp + fp), tp / float(tp + fn)))

tp : 94896, fp : 5190, tn : 958291018, fn : 62549
prec: 0.948144595648, recall: 0.602724761028


##yeast

In [52]:
ft = "/mnt/scratch3/avi/clustering/yeast/cd_hit/0.95.clstr"
tr_clust, tr_clust_inv = readCDHitClust(ft)
ft = "/mnt/scratch3/avi/clustering/yeast/truth/contigs2genes.disambiguous.txt"
groundTruth_clust, ground_truth_clust_inv = readTrueLabels(ft)
tp, fp, tn, fn = accuracyExpressedFast(groundTruth_clust, ground_truth_clust_inv, tr_clust, tr_clust_inv)
print("tp : {}, fp : {}, tn : {}, fn : {}".format(tp, fp, tn, fn))
print("prec: {}, recall: {}".format(tp / float(tp + fp), tp / float(tp + fn)))

tp : 87, fp : 65, tn : 2038467, fn : 571
prec: 0.572368421053, recall: 0.132218844985


In [53]:
ft = "/mnt/scratch3/avi/clustering/yeast/cd_hit/0.8.clstr"
tr_clust, tr_clust_inv = readCDHitClust(ft)
ft = "/mnt/scratch3/avi/clustering/yeast/truth/contigs2genes.disambiguous.txt"
groundTruth_clust, ground_truth_clust_inv = readTrueLabels(ft)
tp, fp, tn, fn = accuracyExpressedFast(groundTruth_clust, ground_truth_clust_inv, tr_clust, tr_clust_inv)
print("tp : {}, fp : {}, tn : {}, fn : {}".format(tp, fp, tn, fn))
print("prec: {}, recall: {}".format(tp / float(tp + fp), tp / float(tp + fn)))

tp : 239, fp : 340, tn : 2038192, fn : 419
prec: 0.412780656304, recall: 0.363221884498
